In [1]:
import httplib2
import json
import sys
import codecs
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np


In [2]:
def getMake():
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMakeID(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    MK_df = pd.DataFrame(result['Results'])
    result_df = MK_df[MK_df['Make_Name'].str.contains(Make)]
    return result_df

def getModel(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/%s?format=json' %Make)
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year(Make,Year):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeYear/make/%s/modelyear/%s?format=json' %(Make,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year_MakeID(MakeID,Year):
    url = ('https://vpic.nhtsa.dot.gov/api//vehicles/GetModelsForMakeIdYear/makeId/%s/modelyear/%s?format=json' %(MakeID,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist
# With score
# def getExactMatch(Make_id,Year):
#     Rlist = []
#     try:
#         Valid_df = getModel_Year_MakeID(Make_id,Year)
        
#     except:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         Rlist=['No Such year in the Database'] *len(QC_Make_year)
#     else:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         for i in range(len(QC_Make_year.Model)):
#             str2Match = str(QC_Make_year.Model.values[i])
#             strOptions = [z for z in Valid_df.Model_Name]
#             Ratios= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
#             Rlist.append((':'.join([ str(a) for a in list(Ratios)])))
#     return Rlist

def getTopMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)
        
    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios= process.extractOne(str2Match,strOptions)
            Rlist.append(( list(Ratios)[0]))
    return Rlist

def getQCtable_OneYear(Make_id,Year):    
    QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
    QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
    QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
    QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
    QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
    QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') for i in range(len(QC_Make_Year['TopMatch']))]
    QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make_Year

def getQCtable_Allyear(Make_id):    
    QC_Make = pd.DataFrame()
    Years = list(ME_df[ME_df.Make.str.lower() == dictionary[Make_id]].Year.unique())
    for Year in Years:
        QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
        QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

# def getHightestScores(Make_id,Year):
#     Scores = []
#     if type(getMatch(Make_id,Year)[0]) !=str:
#         for rec in getMatch(Make_id,Year):
#             Scores.append(int(rec[0].split(': ')[1]))
#     else:
#         Scores='N/A'
#     return Scores




In [3]:
getMakeID('Isuzu')

,Make_ID,Make_Name
4082,542,Isuzu
7323,3760,Spartan/Isuzu


In [ ]:
# def getMake():
#     url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
#     h = httplib2.Http()
#     response, content = h.request(url, 'GET')
#     result = json.loads(content)
#     return pd.DataFrame(result['Results'])

# def getMakeID(Make):
#     url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
#     h = httplib2.Http()
#     response, content = h.request(url, 'GET')
#     result = json.loads(content)
#     MK_df = pd.DataFrame(result['Results'])
#     result_df = MK_df[MK_df['Make_Name'].str.contains(Make)]
#     return result_df

# def getModel(Make):
#     url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/%s?format=json' %Make)
#     h = httplib2.Http()
#     response, content = h.request(url, 'GET')
#     result = json.loads(content)
#     return pd.DataFrame(result['Results'])

# def getModel_Year(Make,Year):
#     url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeYear/make/%s/modelyear/%s?format=json' %(Make,Year))
#     h = httplib2.Http()
#     response, content = h.request(url, 'GET')
#     result = json.loads(content)
#     return pd.DataFrame(result['Results'])

# def getModel_Year_MakeID(MakeID,Year):
#     url = ('https://vpic.nhtsa.dot.gov/api//vehicles/GetModelsForMakeIdYear/makeId/%s/modelyear/%s?format=json' %(MakeID,Year))
#     h = httplib2.Http()
#     response, content = h.request(url, 'GET')
#     result = json.loads(content)
#     return pd.DataFrame(result['Results'])

# def getMatch(Make_id,Year):
#     Rlist = []
#     try:
#         Valid_df = getModel_Year_MakeID(Make_id,Year)

#     except:
#         Rlist.append('No Such year in the Database')
#     else:
#         QC_ford_year = ME_df[(ME_df.Make.str.lower() == 'ford') & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         for i in range(len(QC_ford_year.Model)):
#             str2Match = str(QC_ford_year.Model.values[i])
#             strOptions = [z for z in Valid_df.Model_Name]
#             Ratios = process.extract(str2Match,strOptions)
#             Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
#     return Rlist
# def getQCtable_Allyear(Make_id):    
#     QC_Make = pd.DataFrame()
#     Years = list(ME_df[ME_df.Make.str.lower() == 'ford'].Year.unique())
#     for Year in Years:
#         QC_Make_Year = ME_df[(ME_df.Make.str.lower() == 'ford') & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         QC_Make_Year['Match'] = getMatch(Make_id,Year)
#         QC_Make = pd.concat([QC_Make,QC_Make_Year])
#     return QC_Make
# def getQCtable_OneYear(Make_id,Year):
#     QC_Make_Year = ME_df[(ME_df.Make.str.lower() == 'ford') & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#     QC_Make_Year['Match'] = getMatch(Make_id,Year)
#     return QC_Make_Year


In [7]:
dictionary = {460:'ford',452:'bmw',467 :'chevrolet', 448:'toyota'}
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\F and E Series update 12020219.xlsx', sheet_name = 'MasterEligible', dtype=str)
ME_df

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
0,Acura,CL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
1,Acura,CL,2000,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
2,Acura,CL,2001,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
3,Acura,CL,2002,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
4,Acura,CL,2003,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
5,Acura,CL,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
6,Acura,CL,2004,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
7,Acura,CSX,2005,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8,Acura,CSX,2006,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
9,Acura,CSX,2007,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2


## Use FORD 1992 as an example

In [ ]:
ME_df[ME_df.Make == 'Ford'].sort_values(by=['Make','Year','Model'])

In [ ]:
getMakeID('Ford')

In [ ]:
Ford_id = 460
Vali_df = getModel_Year_MakeID(Ford_id,'1992')
ME_ford_1992 = ME_df[(ME_df.Make == 'Ford') & (ME_df.Year == '2002')].sort_values(by=['Make','Year','Model'])
display(Vali_df)
ME_ford_1992


In [ ]:
str2Match = ME_ford_1992.Model.values[8]
strOptions = [i for i in Vali_df.Model_Name]
print(str2Match)
Ratios = process.extract(str2Match,strOptions,limit =5)
print(dict(Ratios))
list(dict(Ratios))
[a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)]
ME_ford_1992.Model.values[0]
len(ME_ford_1992.Model)

In [ ]:
Make_id = 460

QC_Ford = getQCtable_Allyear(Make_id)

In [ ]:
QC_Ford

In [ ]:
QC_Ford_sorted = QC_Ford.sort_values(by=['Make','Model','Year'])
QC_Ford_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_ford.xlsx',index = False)

In [ ]:
display(getModel('Chevrolet')[getModel('Chevrolet').Model_Name.str.contains('')])
getModel_Year_MakeID(467,'2003')[getModel_Year_MakeID(467,'2003').Model_Name.str.lower().str.contains('')]

## Test Different Scorer

In [ ]:
Rlist = []
Rlist1 = []
Valid_df = getModel_Year_MakeID(460,'2000')
QC_Make_year = ME_df[(ME_df.Make.str.lower() == 'ford') & (ME_df.Year == '2000')].sort_values(by=['Make','Year','Model'])
for i in range(len(QC_Make_year.Model)):
    str2Match = str(QC_Make_year.Model.values[i])
    strOptions = [z for z in Valid_df.Model_Name]
    Ratios = process.extract(str2Match,strOptions)
    Ratios1= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
    Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    Rlist1.append((':'.join([ str(a) for a in list(Ratios1)])))
Rlist1



In [ ]:
getExactMatch(460,'2000')

In [ ]:
len(getMatch(460,'2000')) ==len(getExactMatch(460,'2000'))

In [ ]:
 len(getMatch(460,'1999'))== len(getExactMatch(460,'1999'))

In [ ]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist
def getHightestScores(Make_id,Year):
    Scores = []
    for rec in getMatch(460,'2000'):
        Scores.append(int(rec[0].split(': ')[1]))
    return Scores
len(getHightestScores(460,'2000')) == len(getMatch(460,'2000'))

##  ADD feartures to test


In [10]:
def getQCtable_Allyear(Make_id):    
    QC_Make = pd.DataFrame()
    Years = ['2017']
    for Year in Years:
        QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        QC_Make_Year['TOPexcatMatch'] = getExactMatch(Make_id,Year)
        QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TOPexcatMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TOPexcatMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') for i in range(len(QC_Make_Year['TOPexcatMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make
getQCtable_Allyear(448)

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription,TOPexcatMatch,MatchOptions,Match,HighestScore,TopMaTchExsting
9862,TOYOTA,Aqua,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Sequoia,"[Sequoia: 55, Avalon: 45, Corolla Matrix: 45, ...",No,55,Yes
9974,TOYOTA,C-HR,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Camry,"[Prius C: 86, Corolla: 45, LAND CRUISER: 45, 4...",No,86,Yes
10001,TOYOTA,Corolla,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Corolla,"[Corolla: 100, COROLLA iM: 95, Corolla Matrix:...",Yes,100,Yes
10055,TOYOTA,FJ Cruiser,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,LAND CRUISER,"[LAND CRUISER: 78, 4-Runner: 56, PRIUS: 54, Ya...",No,78,Yes
10080,TOYOTA,Highlander,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Highlander,"[Highlander: 100, LAND CRUISER: 55, Tundra: 45...",Yes,100,Yes
10109,TOYOTA,Land Cruiser,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,LAND CRUISER,"[LAND CRUISER: 100, 4-Runner: 56, Highlander: ...",Yes,100,Yes
10148,TOYOTA,MIRAI,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Mirai,"[Mirai: 100, Corolla Matrix: 54, RAV4: 44, PRI...",Yes,100,Yes
10229,TOYOTA,Prius C,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Prius C,"[Prius C: 100, PRIUS: 95, Prius V: 86, Prius P...",Yes,100,Yes
10358,TOYOTA,Sequoia,2017,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Sequoia,"[Sequoia: 100, Scion iA: 53, Paseo: 50, Sienna...",Yes,100,Yes
9854,Toyota,4-Runner,2017,2019-11-05 00:00:00,Y,round 3 list per Denise,Round 3,4-Runner,"[4-Runner: 100, LAND CRUISER: 56, Tundra: 43, ...",Yes,100,Yes


In [5]:
fuzz.ratio("F 150", "F-150")

80

In [14]:
str2Match = 'this is a test.'
strOptions = ['this is a test!','E-150','F-250']
Ratios = process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
Ratios


('this is a test!', 100)

In [19]:
import Levenshtein as lev
lev.ratio('F-150','F 150')
fuzz.ratio('F-150','F 150')

80

In [ ]:
def getExactMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)
        
    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
            Rlist.append((':'.join([ str(a) for a in list(Ratios)])))
    return Rlist

getExactMatch(460,'2005')

In [24]:
ALL_make = getMake()
ALL_make[ALL_make.Make_ID.astype(str) == '479']

,Make_ID,Make_Name
6407,479,Quest Manufacturing


In [12]:
ALL_make

,Make_ID,Make_Name
0,7756,Bill’s Welding & Repair
1,6473,Blackwater
2,7827,Car Twin Trailers Inc
3,7208,CRUZER
4,7145,Custom Mobile Food Equipment
5,5518,Drake Truck & Trailer Equipment
6,6199,Esterel
7,5527,Gerling and Associates
8,7640,"Indiana Custom Cargo, Inc"
9,5794,Kountry Aire
